In [ ]:
library(tidyverse)
library(glatos)


detections_path <- file.path('../data', 'detections.csv')
detections <- glatos::read_glatos_detections(detections_path)

detections <- glatos::false_detections(detections, tf = 3600)
filtered_detections <- detections %>% filter(passed_filter != FALSE)

detection_events <- glatos::detection_events(filtered_detections, location_col = 'station')
detection_events  

In [ ]:
library(lubridate)

detection_events <- 
    detection_events %>% 
    mutate(detection_interval = lubridate::interval(first_detection, last_detection))

detection_events

In [ ]:
for(event in detection_events$event) {
    detection_events$overlaps_with[event] = paste( # We use paste to create a string of other events
        which(detection_events$location == detection_events$location[event] &  # Make sure that the location is the same
            detection_events$event != event &  # Make sure the event is not the same
            lubridate::int_overlaps(detection_events$detection_interval[event], detection_events$detection_interval) 
            # We can use lubridate's int_overlaps function to find the overlapping events
        ),
        collapse=",")
}

detection_events


In [ ]:
detection_events %>% 
    select(-one_of("detection_interval")) %>% 
    filter(detection_events$overlaps_with != '')  

In [ ]:
summary_data <- 
    detection_events %>% 
    group_by(location) %>% 
    summarise(detection_count = sum(num_detections),
              num_unique_tags = n_distinct(animal_id),
              total_residence_time_in_seconds = sum(detection_interval),
              latitude = mean(mean_latitude),
              longitude = mean(mean_longitude))  

summary_data

In [ ]:
library(plotly)

abacus_plot <-
    filtered_detections %>% 
    filter(!str_detect(station, "lost")) %>% 
    ggplot(aes(x = detection_timestamp_utc, y = animal_id, color = deploy_lat)) +
    geom_point() +
    ylab("Animal ID") + xlab("Date") + labs(color = "Detection latitude") +
    theme_minimal()

## Static plot
abacus_plot

In [ ]:
## Interactive plot using plotly
ggplotly(abacus_plot)

In [ ]:
geo <- list(
  #   scope = 'north america',
  showland = TRUE,
  landcolor = toRGB("#7BB992"),
  showocean = TRUE,
  oceancolor = toRGB("#A0AAB4"),
  showrivers = TRUE,
  rivercolor = toRGB("#A0AAB4"),
  showlakes = TRUE,
  lakecolor = toRGB("#A0AAB4"),
  showcountries = TRUE,
  resolution = 50,
  center = list(lat = ~median(latitude),
                lon = ~median(longitude)),
  lonaxis = list(range=c(~min(longitude) - 4, ~max(longitude) + 4)),
  lataxis = list(range=c(~min(latitude) - 4, ~max(latitude) + 4))
)

In [ ]:
map <- summary_data %>%
    filter(!str_detect(location, "lost")) %>%
    plot_geo(lat = ~latitude, lon = ~longitude, color = ~detection_count, height = 900 )%>%
    add_markers(
        text = ~paste(location, ': ', detection_count,'detections', ' & ', total_residence_time_in_seconds, ' seconds of residence time'),
        hoverinfo = "text",
        size = ~c(detection_count/10)#  + total_residence_time_in_seconds/3600)
    )%>%
    layout(title = "Detections in the Great Lakes", geo = geo)


map  

In [ ]:
Sys.setenv('MAPBOX_TOKEN' = 'pk.eyJ1IjoiYnJ1Y2VkIiwiYSI6ImNrM2Z6NDNscjBhNGYza3AzcW1pZnp3cDQifQ.kQLCJJtGcfX7mvq-wNkr2Q')
#require(devtools)
#install_version("plotly", version = "4.9.0", repos = "http://cran.us.r-project.org")


In [ ]:
mapbox <- summary_data %>%
  filter(!str_detect(location, "lost")) %>%
  plot_mapbox(lat = ~latitude, lon = ~longitude, color = ~detection_count , height = 900) %>%
  add_markers(
    text = ~paste(location, ': ', detection_count,'detections', ' & ', total_residence_time_in_seconds, ' seconds of residence time'),
    hoverinfo = "text",
    size = ~c(detection_count/10  + total_residence_time_in_seconds/3600)
  )%>%
  layout( mapbox = list(zoom = 7,
                        center = list(lat = ~median(latitude),
                                      lon = ~median(longitude))
  ))

mapbox